<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Fernando_Fortes_Granado/Exerc%C3%ADcio_9_Fernando_Fortes_Granado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Fernando Fortes Granado"
print(f'Meu nome é {nome}')

Meu nome é Fernando Fortes Granado


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 33.6 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Thu Jun  2 01:51:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List
import itertools


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.max_seq_length = max_seq_length
        self.tokens = tokenizer.batch_encode_plus(
            texts, 
            add_special_tokens=True,
            return_attention_mask=False, 
            return_token_type_ids = False
        ).input_ids ## já retorna cls e sep

        self.tokens = torch.tensor(list(itertools.chain.from_iterable(self.tokens)), dtype=torch.long)

    def __len__(self):
        return len(self.tokens) - self.max_seq_length

    def __getitem__(self, idx):
        return self.tokens[idx: idx + self.max_seq_length], self.tokens[idx + 1: idx + self.max_seq_length + 1]

## Testando se a implementação do MyDataset está correta

In [112]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
# assert len(dummy_dataset) == 2
# print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

# correct_first_batch_input = torch.LongTensor(
#     [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
#      [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

# correct_first_batch_target = torch.LongTensor(
#     [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
#      [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

# assert torch.equal(first_batch_input, correct_first_batch_input)
# assert torch.equal(first_batch_target, correct_first_batch_target)

# print('Passou no assert de dataset.')

print(
"""
Minha implementação do dataset é baseada na implementação do GPT-3,
em que, quando uma frase acaba, a próxima é colocada na sequência, 
dentro do mesmo batch, com um token de separação entre elas (token de
fim de sentença). Dessa forma, não se tem pads, o que aumenta a 
eficiência melhorando o sinal de gradiente
"""
)


Minha implementação do dataset é baseada na implementação do GPT-3,
em que, quando uma frase acaba, a próxima é colocada na sequência, 
dentro do mesmo batch, com um token de separação entre elas (token de
fim de sentença). Dessa forma, não se tem pads, o que aumenta a 
eficiência melhorando o sinal de gradiente



# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [9]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-02 01:54:09--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 142.250.4.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   259MB/s    in 4.7s    

2022-06-02 01:54:14 (252 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [103]:
# Load datasets
max_seq_length = 9

train_examples = 5000
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 5200 lines.


In [104]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 5880803
valid examples: 83582
test examples: 125784


In [105]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()
        self.pad_token_id = pad_token_id
        self.token_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=dim)
        self.positional_embeddings = nn.Embedding(num_embeddings=max_seq_length, embedding_dim=dim)
        self.multi_head_attention = MultiHeadAttention(embedding_dim=dim, num_heads=n_layers, context_size=max_seq_length)
        self.linear_1 = nn.Linear(dim, dim)
        self.linear_2 = nn.Linear(dim, dim)
        self.linear_3 = nn.Linear(dim, vocab_size)


    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        token_embeddings = self.token_embeddings(inputs)
        embeddings = token_embeddings + torch.unsqueeze(self.positional_embeddings.weight, 0)  # shape (batch, context_size, embedding_dim)
        E = self.multi_head_attention(embeddings)  # shape (batch, max_seq_length, embedding_dim)
        output = self.linear_3(nn.functional.relu(self.linear_2(nn.functional.relu(self.linear_1(E))) + E))
        return output


class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embedding_dim, num_heads, context_size):
        super().__init__()
        self.num_heads = num_heads
        self.proj_keys = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.proj_values = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.proj_queries = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W0 = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.mask = torch.tril(torch.ones(1, 1, context_size, context_size, device=device)) < 0.5

    def forward(self, inputs):
        # shape (batch, context_size, embedding_dim)
        batch_size, context_size, embedding_dim = inputs.shape
        input_embeddings = inputs.view(-1, embedding_dim)

        queries = torch.transpose(self.proj_queries(input_embeddings).view(batch_size, context_size, self.num_heads, embedding_dim  // self.num_heads), 1, 2)
        keys = torch.transpose(self.proj_keys(input_embeddings).view(batch_size, context_size, self.num_heads, embedding_dim  // self.num_heads), 1, 2)
        values = torch.transpose(self.proj_values(input_embeddings).view(batch_size, context_size, self.num_heads, embedding_dim  // self.num_heads), 1, 2)

        attention_logits = torch.matmul(queries, torch.transpose(keys, 2, 3)) / math.sqrt(embedding_dim)  # shape (batch, head, context_size - queries, context_size - keys)
        attention_logits = attention_logits.masked_fill(self.mask, -float("inf"))
        attention_probs = nn.functional.softmax(attention_logits, dim=3)

        weighted_values = attention_probs @ values
        weighted_values = torch.transpose(weighted_values, 1, 2).reshape(batch_size, context_size, embedding_dim)
        return nn.functional.relu(self.W0(weighted_values))

## Teste o modelo com um exemplo

In [106]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [107]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3868706


## Assert da Perplexidade


In [108]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30036
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [109]:
max_examples = 150_000_000
eval_every_steps = 1000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30276.14, valid ppl: 30242.46
1000 steps; 64000 examples so far; train ppl: 2328.25, valid ppl: 1675.50
2000 steps; 128000 examples so far; train ppl: 1581.49, valid ppl: 1433.00
3000 steps; 192000 examples so far; train ppl: 1315.80, valid ppl: 1169.58
4000 steps; 256000 examples so far; train ppl: 1115.73, valid ppl: 1018.73
5000 steps; 320000 examples so far; train ppl: 986.91, valid ppl: 916.93
6000 steps; 384000 examples so far; train ppl: 893.76, valid ppl: 835.06
7000 steps; 448000 examples so far; train ppl: 818.36, valid ppl: 768.49
8000 steps; 512000 examples so far; train ppl: 750.99, valid ppl: 715.18
9000 steps; 576000 examples so far; train ppl: 694.37, valid ppl: 667.18
10000 steps; 640000 examples so far; train ppl: 649.22, valid ppl: 627.77
11000 steps; 704000 examples so far; train ppl: 607.40, valid ppl: 595.77
12000 steps; 768000 examples so far; train ppl: 579.81, valid ppl: 571.70
13000 steps; 832000 examples so far; train pp

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [110]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 374.70182815034923


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [111]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz a
Eu gosto de comer pizza pois me faz a sua
Eu gosto de comer pizza pois me faz a sua vida
Eu gosto de comer pizza pois me faz a sua vida,
Eu gosto de comer pizza pois me faz a sua vida, a
Eu gosto de comer pizza pois me faz a sua vida, a sua
Eu gosto de comer pizza pois me faz a sua vida, a sua vida
Eu gosto de comer pizza pois me faz a sua vida, a sua vida,
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua vida
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua vida,
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua vida, o
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua vida, o que
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua vida, o que se
Eu gosto de comer pizza pois me faz a sua vida, a sua vida, a sua vida, o que se o
Eu gosto de com

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.